In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
from yaml import safe_load

In [2]:
from StringProcessing import preprocessInput, encode_pad_tag
from SiameseModel import ContrastiveLoss, CosineDistance
from ModelEvaluation import find_Model_ranks

tf.version:  2.9.1
tf.keras.version:  2.9.0
tf.config.devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-09-15 12:16:29.758476: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 12:16:29.784010: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 12:16:29.784163: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 12:16:29.785366: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

In [3]:
with open('LSTM65.yaml', 'r') as file:
    model_config = safe_load(file)

In [4]:
model = tf.keras.models.load_model(
    './saved_models/{}'.format(model_config["Model_Name"]), 
    custom_objects = {
        'CosineDistance': CosineDistance,
        'ContrastiveLoss': ContrastiveLoss
    }, 
    compile=True, 
    options=None
)
model.summary()

2022-09-15 12:16:30.283936: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 12:16:30.284169: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 12:16:30.284275: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 12:16:30.284411: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 12:16:30.284517: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

2022-09-15 12:16:36.374437: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-09-15 12:16:36.385784: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-09-15 12:16:36.828925: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-09-15 12:16:36.839676: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-09-15 12:16:36.958488: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 13 outputs but the _output_sh

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 65)]         0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 65)]         0           []                               
                                                                                                  
 model (Functional)             (None, 180)          168300      ['input_2[0][0]',                
                                                                  'input_3[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 1)            0           ['model[0][0]',            

In [5]:
unmapped_sequences = encode_pad_tag(preprocessInput("../data/unmapped-faers-drug-strings-Q12004-through-Q2-2021-extended-NP-vocab-09012022.tsv", maxlen= model_config['MAXLEN'], balance=False, reflexive = False, sep="\t"), match = 1, distance = 0, maxlen = model_config['MAXLEN'] )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 389386 entries, 0 to 389385
Data columns (total 1 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   drug_name_original  389385 non-null  object
dtypes: object(1)
memory usage: 3.0+ MB
None
Processing file: ----------------------------------------
Renaming colums:
	Current names: Index(['drug_name_original'], dtype='object')
	New names: Index(['x'], dtype='object')
Dropping row with empty cells:
	Dropped 1 rows with empty cells.
	Cleaning string sequences.
	Uppercasing string sequences.
Dropping sequences longer than the maxlen of 65:
	Dropped 7751 that exceeded the maximum sequence length.
Dropping duplicate sequences:
	Dropped 41849 duplicate sequences.
Done processing: ---------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339785 entries, 0 to 339784
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  -------

In [6]:
dUnique_df = pd.read_pickle("../data/dUnique_df.pkl")

In [7]:
latin_2_common = preprocessInput('../data/vocabulary_LB_to_common_names_09082022.tsv', maxlen= model_config['MAXLEN'], reflexive=True, balance = False, sep='\t')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1299 entries, 0 to 1298
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   lb        1299 non-null   object
 1   mappings  1299 non-null   object
dtypes: object(2)
memory usage: 20.4+ KB
None
Processing file: ----------------------------------------
Renaming colums:
	Current names: Index(['lb', 'mappings'], dtype='object')
	New names: Index(['x', 'y'], dtype='object')
Dropping row with empty cells:
	Dropped 0 rows with empty cells.
	Cleaning string sequences.
	Uppercasing string sequences.
Dropping sequences longer than the maxlen of 65:
	Dropped 0 that exceeded the maximum sequence length.
Dropping duplicate sequences:
	Dropped 321 duplicate sequences.
Done processing: ---------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2277 entries, 0 to 2276
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------

In [8]:
common_2_latin = preprocessInput('../data/positive-unmapped-pairs-common-name-or-latin-binomial-copies.tsv', maxlen= model_config['MAXLEN'], reflexive=True, balance = False, sep='\t')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728 entries, 0 to 727
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   common_name     728 non-null    object
 1   latin_binomial  728 non-null    object
dtypes: object(2)
memory usage: 11.5+ KB
None
Processing file: ----------------------------------------
Renaming colums:
	Current names: Index(['common_name', 'latin_binomial'], dtype='object')
	New names: Index(['x', 'y'], dtype='object')
Dropping row with empty cells:
	Dropped 0 rows with empty cells.
	Cleaning string sequences.
	Uppercasing string sequences.
Dropping sequences longer than the maxlen of 65:
	Dropped 0 that exceeded the maximum sequence length.
Dropping duplicate sequences:
	Dropped 728 duplicate sequences.
Done processing: ---------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728 entries, 0 to 727
Data columns (total 2 columns):
 #   Column  Non-Null Count 

In [9]:
gsrs_2_common = preprocessInput('../data/positive-unmapped-pairs-gsrs-name-to-common-name.tsv', maxlen= model_config['MAXLEN'], reflexive=True, balance = False, sep='\t')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10838 entries, 0 to 10837
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         10838 non-null  object
 1   common_name  199 non-null    object
dtypes: object(2)
memory usage: 169.5+ KB
None
Processing file: ----------------------------------------
Renaming colums:
	Current names: Index(['name', 'common_name'], dtype='object')
	New names: Index(['x', 'y'], dtype='object')
Dropping row with empty cells:
	Dropped 10639 rows with empty cells.
	Cleaning string sequences.
	Uppercasing string sequences.
Dropping sequences longer than the maxlen of 65:
	Dropped 0 that exceeded the maximum sequence length.
Dropping duplicate sequences:
	Dropped 0 duplicate sequences.
Done processing: ---------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  --

In [36]:
equivalence_set = pd.concat([latin_2_common, common_2_latin, gsrs_2_common], ignore_index=True)
equivalence_set.drop_duplicates(subset=['x', 'y'], inplace=True)

In [37]:
results = find_Model_ranks(model, df = unmapped_sequences.sample(n=300), ranks = 20, targets= dUnique_df, equivalence_set = equivalence_set)

In [38]:
results = results.iloc[:, ~results.columns.isin(['Processed_x', 'Match', 'Distance'])]

In [39]:
results.head(3)

,x,rank1,rank2,rank3,rank4,rank5,rank6,rank7,rank8,rank9,...,rank11,rank12,rank13,rank14,rank15,rank16,rank17,rank18,rank19,rank20
15244,ZICAM ALLERGY RELIEF NASAL GEL A,AAI,HELICHRYSUM ARENARIUM,HONG JING TIAN,ILEX PARAGUARIENSIS,ZINGIBER OFFICINALE,BALSAMAPPLE,BALLOTA NIGRA,BALLOTA,KAWA PEPPER,...,KOLA,KRATOM,KRATUM,LIAN,LOQUAT,MALVA SYLVESTRIS,MARRUBIUM NIGRUM,MELIA AZEDARACH,PIPER NIGRUM,PIPER METHYSTICUM
262137,SHUANGHUANGLIAN ML,COMMON VALERIAN,HYDRASTIS CANADENSIS,SHAN ZHU YU,GOLDENSEAL,CEYLON CINNAMON,HUANGLIAN,HUANG LIAN,SQUAW BALM,FIELD BALM,...,PHYLLANTHUS AMARUS,YELLOW ROOT,SUAN CHENG,CHUI XU SHANG LU,COPTIS CHINENSIS,SPINACH CHARD,HYDRASTIS CANADENSIS L,JUGLANS REGIA,ZHANG YE DA HUANG,HELICHRYSUM ARENARIUM
119707,RINDERON OPHTHALMIC SOLUTION,YAWROOT,BUTCHERSBROOM,INDIGOFERA HIRSUTA,DA HUA LONG YA CAO,WOODLAND HAWTHORN,GINGER,LAPACHO NEGRO,DANDELION,FRENCH LILAC,...,PAPOOSEROOTT,HANDROANTHUS HEPTAPHYLLUS,ROTAHORN,JOINTED CHARLOCK,AGRIMONIA EUPATORIA,LAURIER TULIPIER,ZANTHOXYLUM CLAVA,POGOSTEMON CABLIN,STILLINGIA SYLVATICA,TRIGONELLA FOENUM


In [40]:
for i in results.index:
    results.loc[i].to_csv("./Novelty_Results/{}.csv".format(i), sep='\n', header= False, index=False)